In [1]:
from tqdm.auto import tqdm
from call_model import call_chat_once
import asyncio
import httpx
from uuid import uuid4
import json
import datasets
from typing import Optional
import os
import csv
import nest_asyncio
nest_asyncio.apply()

/home/minhthuy/anaconda3/envs/chatbot/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
eval_dataset = datasets.load_dataset("m-ric/huggingface_doc_qa_eval", split="train") # or load from data/test.csv
print(eval_dataset)

Using the latest cached version of the dataset since m-ric/huggingface_doc_qa_eval couldn't be found on the Hugging Face Hub
2025-08-13 17:25:46,310 - WARNING - Using the latest cached version of the dataset since m-ric/huggingface_doc_qa_eval couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /home/minhthuy/.cache/huggingface/datasets/m-ric___huggingface_doc_qa_eval/default/0.0.0/5f70aa9a1e2430f528ac3f27f01f0ba8719c0704 (last modified on Mon Aug 11 14:41:56 2025).
2025-08-13 17:25:46,318 - WARNING - Found the latest cached dataset configuration 'default' at /home/minhthuy/.cache/huggingface/datasets/m-ric___huggingface_doc_qa_eval/default/0.0.0/5f70aa9a1e2430f528ac3f27f01f0ba8719c0704 (last modified on Mon Aug 11 14:41:56 2025).


Dataset({
    features: ['context', 'question', 'answer', 'source_doc', 'standalone_score', 'standalone_eval', 'relatedness_score', 'relatedness_eval', 'relevance_score', 'relevance_eval'],
    num_rows: 65
})


In [3]:
OUTPUT_FILE = "data/rag_results_default_as_retriever.csv"

def load_existing_results(file_path):
    dataset = []
    existing_questions = set()
    if os.path.exists(file_path):
        with open(file_path, newline='', encoding='utf-8') as f:
            reader = csv.DictReader(f)
            for row in reader:
                existing_questions.add(row["user_input"])
                # parse retrieved_contexts lại từ JSON
                row["retrieved_contexts"] = json.loads(row["retrieved_contexts"])
                dataset.append(row)
    return existing_questions, dataset


def append_result_to_csv(file_path, row, fieldnames):
    file_exists = os.path.exists(file_path)
    with open(file_path, "a", newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        if not file_exists:
            writer.writeheader()
        writer.writerow(row)

def run_rag_tests(
    eval_dataset: datasets.Dataset,
    verbose: Optional[bool] = False
):
    """Runs RAG tests on the given dataset and saves the results to the given output file."""
    fieldnames = ["user_input", "retrieved_contexts", "response", "reference"]

    processed_questions, dataset = load_existing_results(OUTPUT_FILE)
    print(f"Đã có {len(processed_questions)} câu hỏi xử lý trước đó, sẽ bỏ qua chúng.")


    for i, example in enumerate(tqdm(eval_dataset)):
        question = example["question"]

        if question in processed_questions:
            continue
        
        payload = {
            "question": question,
            "session_id": str(uuid4()),
            "chat_history": []
        }

        try:
            relevant_docs, answer = asyncio.run(call_chat_once(payload))
        except Exception as e:
            print(f"Error from question: '{question}': {e}")

        if verbose:
            print("=======================================================")
            print(f"Question: {question}")
            print(f"Answer: {answer}")
            print(f'True answer: {example["answer"]}')

        row = {
            "user_input": question,
            "retrieved_contexts": json.dumps(
                [doc['kwargs'].get('page_content') for doc in relevant_docs],
                ensure_ascii=False
            ),
            "response": answer,
            "reference": example['answer']
        }
        append_result_to_csv(OUTPUT_FILE, row, fieldnames)
        dataset.append({
            "user_input": question,
            "retrieved_contexts": json.loads(row["retrieved_contexts"]),
            "response": answer,
            "reference": example['answer']
        })
    return dataset

In [4]:
data = run_rag_tests(eval_dataset=eval_dataset)


Đã có 0 câu hỏi xử lý trước đó, sẽ bỏ qua chúng.


  2%|▏         | 1/65 [00:03<03:51,  3.62s/it]

{'messages': [{'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'messages', 'HumanMessage'], 'kwargs': {'content': 'What architecture is the `tokenizers-linux-x64-musl` binary designed for?\n', 'type': 'human', 'id': '8be83939-ff1b-4cd9-bfb8-d771f2b64c66'}}, {'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'messages', 'AIMessage'], 'kwargs': {'content': '', 'additional_kwargs': {'tool_calls': [{'index': 0, 'id': 'call_iTUG5dqzmzRzV2QxuHLXepUd', 'function': {'arguments': '{"query":"tokenizers-linux-x64-musl architecture"}', 'name': 'data_retriever'}, 'type': 'function'}]}, 'response_metadata': {'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c'}, 'type': 'ai', 'id': 'run--0a6bf289-050a-43ea-b8c5-ca9fff42ea01-0', 'tool_calls': [{'name': 'data_retriever', 'args': {'query': 'tokenizers-linux-x64-musl architecture'}, 'id': 'call_iTUG5dqzmzRzV2QxuHLXepUd', 'type': 'tool_call'}], 'usage_metadata': {'input_tok

2025-08-13 17:25:56,109 - INFO - HTTP Request: POST http://localhost:8001/api/v1/chat-evaluation "HTTP/1.1 200 OK"
  3%|▎         | 2/65 [00:09<05:19,  5.07s/it]

{'messages': [{'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'messages', 'HumanMessage'], 'kwargs': {'content': 'What is the purpose of the BLIP-Diffusion model?\n', 'type': 'human', 'id': 'cf0a1dd1-451c-4ba9-8a26-a76867c62d17'}}, {'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'messages', 'AIMessage'], 'kwargs': {'content': '', 'additional_kwargs': {'tool_calls': [{'index': 0, 'id': 'call_30gknCmeGQtVnVm3cQNcGgOw', 'function': {'arguments': '{"query":"BLIP-Diffusion model purpose"}', 'name': 'data_retriever'}, 'type': 'function'}]}, 'response_metadata': {'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c'}, 'type': 'ai', 'id': 'run--795b3785-55a8-46de-86b7-f415bdb11009-0', 'tool_calls': [{'name': 'data_retriever', 'args': {'query': 'BLIP-Diffusion model purpose'}, 'id': 'call_30gknCmeGQtVnVm3cQNcGgOw', 'type': 'tool_call'}], 'usage_metadata': {'input_tokens': 361, 'output_tokens': 22, 'total_tokens

...

{'messages': [{'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'messages', 'HumanMessage'], 'kwargs': {'content': 'What is the name of the open-source library created by Hugging Face to simplify Transformer acceleration?\n', 'type': 'human', 'id': 'f24c4dc2-e30d-434e-bf4b-3d75c21433e3'}}, {'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'messages', 'AIMessage'], 'kwargs': {'content': '', 'additional_kwargs': {'tool_calls': [{'index': 0, 'id': 'call_V5E50znKeNYlbMvcoE2va0XG', 'function': {'arguments': '{"query":"open-source library created by Hugging Face to simplify Transformer acceleration"}', 'name': 'data_retriever'}, 'type': 'function'}]}, 'response_metadata': {'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c'}, 'type': 'ai', 'id': 'run--af94b493-4f08-456b-bdd2-5d9e6c081bca-0', 'tool_calls': [{'name': 'data_retriever', 'args': {'query': 'open-source library created by Hugging Face to simplify Tra

2025-08-13 17:30:12,595 - INFO - HTTP Request: POST http://localhost:8001/api/v1/chat-evaluation "HTTP/1.1 200 OK"
 98%|█████████▊| 64/65 [04:26<00:03,  3.73s/it]

{'messages': [{'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'messages', 'HumanMessage'], 'kwargs': {'content': 'What parameter is used to ensure that elements in a row have the same height in Gradio?\n', 'type': 'human', 'id': 'c46c26cc-682e-4852-9b62-1f09fcc42aae'}}, {'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'messages', 'AIMessage'], 'kwargs': {'content': '', 'additional_kwargs': {'tool_calls': [{'index': 0, 'id': 'call_GimwLOegVFbVwvbOEah7NmHD', 'function': {'arguments': '{"query":"Gradio same height elements in a row parameter"}', 'name': 'data_retriever'}, 'type': 'function'}]}, 'response_metadata': {'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c'}, 'type': 'ai', 'id': 'run--279307b7-456d-4af0-8e28-fcc608eeb70d-0', 'tool_calls': [{'name': 'data_retriever', 'args': {'query': 'Gradio same height elements in a row parameter'}, 'id': 'call_GimwLOegVFbVwvbOEah7NmHD', 'type': 'tool_call'}],

2025-08-13 17:30:16,117 - INFO - HTTP Request: POST http://localhost:8001/api/v1/chat-evaluation "HTTP/1.1 200 OK"
100%|██████████| 65/65 [04:29<00:00,  4.15s/it]

{'messages': [{'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'messages', 'HumanMessage'], 'kwargs': {'content': 'What is the command to install the latest version of Optimum with OpenVINO support?\n', 'type': 'human', 'id': 'cf695375-ecba-47b7-bf4e-778cda1a0181'}}, {'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'messages', 'AIMessage'], 'kwargs': {'content': '', 'additional_kwargs': {'tool_calls': [{'index': 0, 'id': 'call_6Aw9FmAouJ3wTnVLE6TmGu6o', 'function': {'arguments': '{"query":"install latest version of Optimum with OpenVINO support"}', 'name': 'data_retriever'}, 'type': 'function'}]}, 'response_metadata': {'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c'}, 'type': 'ai', 'id': 'run--f3f54d28-905e-4c11-82c6-250680f52763-0', 'tool_calls': [{'name': 'data_retriever', 'args': {'query': 'install latest version of Optimum with OpenVINO support'}, 'id': 'call_6Aw9FmAouJ3wTnVLE6TmGu6o', 'type': 

In [5]:
from ragas import EvaluationDataset
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
    FactualCorrectness
)
evaluation_dataset = EvaluationDataset.from_list(data)

# Khởi tạo các metric
metrics = [
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
]
# metrics = [FactualCorrectness()]

In [6]:
print(len(evaluation_dataset))

65


In [ ]:
from app.config import configs

import os
os.environ['OPENAI_API_KEY'] = configs.OPENAI_API_KEY

result = evaluate(
    evaluation_dataset,
    metrics=metrics,
)

print(result) 

Evaluating:   0%|          | 0/260 [00:00<?, ?it/s]2025-08-13 17:30:19,680 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-13 17:30:19,745 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-13 17:30:19,748 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-13 17:30:19,780 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-13 17:30:19,816 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-13 17:30:19,829 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-13 17:30:19,867 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-13 17:30:19,870 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-13 17:30:20,716 - INFO - HTTP Request